In [8]:
class Packet():
    
    def __init__(self):
        self.name = 'N/A'
        self.version = 0
        self.typ = 0
        self.value = -1
        self.valstr = ''
        self.packets = []
        
    def sum_versions(self):
        s = self.version
        for p in self.packets:
            s += p.sum_versions()
        return s
    
    def get_results(self):
        if self.typ == 0:
            val = 0
            for v in [p.get_results() for p in self.packets]:
                val += v
            return val

        elif self.typ == 1:
            res = 1
            for v in [p.get_results() for p in self.packets]:
                res *= v
            return res
            
        elif self.typ == 2:
            return min([p.get_results() for p in self.packets])

        elif self.typ == 3:
            return max([p.get_results() for p in self.packets])

        elif self.typ == 4:
            return self.value

        elif self.typ == 5:
            if len(self.packets) != 2:
                # print(len(self.packets))
                raise Exception
            if self.packets[0].get_results() > self.packets[1].get_results():
                return 1
            else:
                return 0

        elif self.typ == 6:
            if len(self.packets) != 2:
                # print(len(self.packets))
                raise Exception
            if self.packets[0].get_results() < self.packets[1].get_results():
                return 1
            else:
                return 0

        elif self.typ == 7:
            if len(self.packets) != 2:
                # print(len(self.packets))
                raise Exception
            if self.packets[0].get_results() == self.packets[1].get_results():
                return 1
            else:
                return 0


    def __repr__(self):
        s = '{n}(V: {v}, T: {t}, P: {p}, Val: {vl} ({vs}))'.format(n=self.name, v=self.version,
                                                                                   t=self.typ, p=self.packets, 
                                                                                   vl=self.value, vs=self.valstr)
        return s

In [15]:
class Operator(Packet):
    
    def __init__(self, data, idx=0):
        super().__init__()

        self.name = 'Operator'
        if data[idx+3:idx+6] == '100':
            raise Exception
            
        #print('Parsing operator from', data)
        # print('Parsing operator')
            
        self.version = int(data[idx:idx+3], 2)
        self.typ = int(data[idx+3:idx+6], 2)
        
        # print(data[idx+6])
        if data[idx+6] == '0':
            n_bits = int(data[idx+7:idx+22], 2)
            # print('n_bits', n_bits)
            idx = idx + 22
            data_end = idx+n_bits
            while '1' in data[idx:data_end]:
                # print('reading')
                p = read_packet(data[idx:data_end])
                self.packets.append(p)
                idx = idx+p.end
                # print('idx now', idx, data[idx:])
        else:
            n_packets = int(data[idx+7:idx+18], 2)
            # print('n_packets', n_packets)
            
            idx = idx+18
            for i in range(0, n_packets):
                p = read_packet(data[idx:])
                self.packets.append(p)
                idx = idx+p.end

        # print('Done')
        self.end = idx

        # if (idx-5) % 5 != 0:
        #    # Remove padding
        #    idx = (int((idx-5) / 5)*5+10)

        #self.padded_end = idx

In [16]:
def read_packet(data):
    if data[3:6] == '100':
        return LiteralValue(data)
    else:
        return Operator(data)


class LiteralValue(Packet):
    
    def __init__(self, data):
        super().__init__()
        
        self.name = 'Value'
        if data[3:6] != '100':
            raise Exception
            
        # print('Parsing value from', data)
        # print('Parsing value')
        self.version = int(data[0:3], 2)
        self.typ = int(data[3:6], 2)
        
        idx = 6
        self.valstr = ''
        packet_done = False
        
        while not packet_done:
            # print(idx, data[idx])
            # Parse the packets
            if data[idx] == '0':
                packet_done = True
            # print(data[idx+1:idx+5])
            self.valstr += data[idx+1:idx+5]
            
            idx += 5
            
        self.value = int(self.valstr, 2)
        #if (idx-5) % 5 != 0:
        #    # Remove padding
        #    idx = (int((idx-5) / 5)*5+10)

        self.end = idx

In [17]:
data = ''
with open('input16.txt', 'r') as inf:
    line = inf.readline()
    for c in line.strip():
        data += format(int(c, 16), '04b')
            
            
res = read_packet(data)

print(res.sum_versions())

print(res.get_results())


977
101501020883
